In [ ]:
%matplotlib notebook

import json
import matplotlib.pyplot as plt
import matplotlib.patches
import numpy as np
import sklearn as sk

from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSCanonical

np.set_printoptions(precision=3, linewidth=100)


In [ ]:
def rebuild_covariance_matrix(v):
    cov = np.zeros((6,6))
    cursor = 0
    for i in range(6):
        cov[i, 0:(i+1)] = v[cursor:(cursor + i + 1)]
        cursor += i + 1
        
    cov = np.dot(cov, cov.T)
        
    return cov

In [ ]:
with open('/home/dlandry/dataset/recov/with_more_data.json') as dataset_file:
    dataset_dict = json.load(dataset_file)
    
xs = np.array(dataset_dict['data']['xs'])
ys = np.array(dataset_dict['data']['ys'])
merged = np.hstack([xs,ys])
scaled = scale(merged)

In [ ]:
pca = PCA(n_components=6)
pca.fit(scaled)

In [ ]:
idx = np.arange(len(pca.components_[0]))

plt.bar(idx, pca.components_[0], 0.2)
plt.bar(idx + 0.2, pca.components_[1], 0.2)
plt.bar(idx + 0.4, pca.components_[2], 0.2)
plt.show()

In [ ]:
idx = np.arange(len(xs))
np.random.shuffle(idx)
training_size = int(len(xs) * 0.8)

xs_training = xs[idx[0:training_size]]
ys_training = ys[idx[0:training_size]]

xs_validation = xs[idx[training_size:]]
ys_validation = ys[idx[training_size:]]

In [ ]:
pls = PLSCanonical(n_components=6, scale=True)
pls.fit(xs_training,ys_training)

In [ ]:
ys_predicted = pls.predict(xs_validation)
loss_matrix = ys_validation - ys_predicted
norm = np.linalg.norm(loss_matrix, axis=1)
avg_loss = np.mean(norm)
print(avg_loss)

In [ ]:
rebuild_covariance_matrix(ys_predicted[20])

In [ ]:
rebuild_covariance_matrix(ys_validation[20])

In [ ]:
def plot_covariance_matrix(c, dims, ax, color='black'):
    submatrix = (c[list(dims)])[:,list(dims)]
    eigvals, eigvecs = np.linalg.eig(submatrix)
    
    angle = np.arctan2(eigvecs[0][1], eigvecs[0][0])
    
    ell = matplotlib.patches.Ellipse((0., 0.), np.sqrt(eigvals[0]), np.sqrt(eigvals[1]), np.rad2deg(angle),
                                    linewidth=1.0, edgecolor=color, fill=False)
    
    ax.add_artist(ell)
    

In [ ]:
def compare_covariance_matrices(c1, c2):
    fig, ax = plt.subplots()
    plot_covariance_matrix(c1, (0,1), ax)
    plot_covariance_matrix(c2, (0,1), ax)
    plt.show()

In [ ]:
fig, ax = plt.subplots()

to_compare = 16

plot_covariance_matrix(rebuild_covariance_matrix(ys_validation[to_compare]), (0,1), ax, color='0.0')
plot_covariance_matrix(rebuild_covariance_matrix(ys_predicted[to_compare]), (0,1), ax, color='0.5')

ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)

